In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
from __future__ import annotations

import logging
from time import time
import requests
import uuid
from google.protobuf.json_format import MessageToDict
from typing import Callable, Optional, Union
import random
import vega_sim.grpc.client as vac
import vega_sim.proto.data_node.api.v1 as data_node_protos
import vega_sim.proto.vega as vega_protos
import vega_sim.api.faucet as faucet
from vega_sim.api.helpers import get_enum, enum_to_str, wait_for_acceptance

import vega_sim.proto.vega.api.v1.core_pb2 as core_proto
import vega_sim.proto.vega.commands.v1.commands_pb2 as commands_proto
import vega_sim.proto.vega.commands.v1.transaction_pb2 as transaction_proto
import vega_sim.proto.vega.commands.v1.signature_pb2 as signature_proto

from nacl.signing import SigningKey
from nacl.encoding import HexEncoder, RawEncoder
import nacl.hash
import hashlib
import numpy as np

In [3]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

# Set up parties in the market/ Submit liquidity provision/ Control midprice
MM_WALLET = WalletConfig("mm", "pin")

# The party to send selling/buying MOs to hit LP orders
TRADER_WALLET = WalletConfig("Zl3pLs6Xk6SwIK7Jlp2x", "bJQDDVGAhKkj3PVCc7Rr")

# The party randomly post LOs at buy/sell side to simulate real Market situation
RANDOM_WALLET = WalletConfig("OJpVLvU5fgLJbhNPdESa", "GmJTt9Gk34BHDlovB7AJ")

# The party to terminate the market and send settlment price
TERMINATE_WALLET = WalletConfig("FJMKnwfZdd48C8NqvYrG", "bY3DxwtsCstMIIZdNpKs")

wallets = [MM_WALLET, TRADER_WALLET, RANDOM_WALLET, TERMINATE_WALLET]

In [63]:
vega = VegaServiceNull(run_wallet_with_console=False)
vega.start()

/var/folders/yj/cjhtlxn90wldd1hvw5lkxnrc0000gn/T/tmp28rl0ru1


In [64]:
for wallet in wallets:
    vega.create_wallet(wallet.name, wallet.passphrase)

In [65]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=100000,
)

vega.forward("10s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=1e10,
)

In [66]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [67]:

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=1e5
)
vega.mint(
    TERMINATE_WALLET.name,
    tdai_id,
    amount=1e2
)
vega.mint(
    RANDOM_WALLET.name,
    tdai_id,
    amount=1e5
)

In [68]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
    )

In [69]:
market_id = vega.all_markets()[0].id

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [70]:
market_id

'3ce7037c56f04bf9212e839412fe6e5ee634065cb5d74255658690f284449a6e'

In [71]:
login_token = vega.login_tokens[MM_WALLET.name]
data_client = vega.trading_data_client()
pub_key = vega.pub_keys[MM_WALLET.name]

In [84]:
headers = {"Authorization": f"Bearer {login_token}"}

order_ref = f"{pub_key}-{uuid.uuid4()}"

order_data = vega_protos.commands.v1.commands.OrderSubmission(
    market_id=market_id,
    # price is an integer. For example 123456 is a price of 1.23456,
    # assuming 5 decimal places.
    price="10",
    side=vega_protos.vega.Side.SIDE_BUY,
    size=10,
    time_in_force=vega_protos.vega.Order.TimeInForce.TIME_IN_FORCE_GTC,
    type=vega_protos.vega.Order.Type.TYPE_LIMIT,
    reference=order_ref,
)

In [73]:
submission = {
        "orderSubmission": MessageToDict(order_data),
        "pubKey": pub_key,
        "propagate": False,
    }

In [74]:
url = f"{vega.wallet_url()}/api/v1/command/sync"

In [75]:
enc = str(submission).encode()

In [90]:
%%timeit
response = requests.post(url, headers=headers, json=submission)
# response.raise_for_status()

1.32 ms ± 18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [36]:
sk = SigningKey.generate()

In [37]:
pub_key = sk.verify_key.encode(encoder=HexEncoder).decode()

In [81]:
faucet.mint(sk.verify_key.encode(encoder=HexEncoder).decode(), tdai_id, 10000, vega.faucet_url())

In [39]:
block_height = vega.core_client().LastBlockHeight(core_proto.LastBlockHeightRequest()).height

In [40]:
%%timeit
sk.sign(enc, encoder=HexEncoder)

72.4 µs ± 538 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [41]:

def count_zeros(b):
    ret = 0
    for x in b:
        if x == 0:
            ret += 8
        else:
            if x % 128 != 0:
                break
            elif x % 64 != 0:
                ret += 1
                break
            elif x % 32 != 0:
                ret += 2
                break
            elif x % 16 != 0:
                ret += 3
                break
            elif x % 8 != 0:
                ret += 4
                break
            elif x % 4 != 0:
                ret += 5
                break
            elif x % 2 != 0:
                ret += 6
                break
            elif x % 1 != 0:
                ret += 7
                break
            break
    return ret

In [42]:
nonces = [(nonce).to_bytes(2, byteorder='big') for nonce in range(0, 10000)]
def PoW(signed_data, tx_id):
    valid_hash = None
    for nonce in nonces:
        new_key = b"Vega_SPAM_PoW" + signed_data + tx_id + nonce
        hashed = hashlib.sha256()
        hashed.update(new_key)
        hashed_dig = hashed.digest()
        if count_zeros(hashed_dig) >= 1:
            valid_hash = hashed_dig
            break
    return nonce, valid_hash

In [252]:
%%timeit
block_height = vega.core_client().LastBlockHeight(core_proto.LastBlockHeightRequest()).height
signed = sk.sign(enc, encoder=HexEncoder)
tx_id = random.getrandbits(10*8).to_bytes(10, byteorder='big')
PoW(signed, tx_id)

515 µs ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [115]:
import time
time.perf_counter?

Docstring:
perf_counter() -> float

Performance counter for benchmarking.
Type:      builtin_function_or_method


In [128]:
%%timeit
block_height = vega.core_client().LastBlockHeight(core_proto.LastBlockHeightRequest()).height
signed = sk.sign(enc, encoder=HexEncoder)


tx_id = random.getrandbits(10*8)
# tx_id_bytes = tx_id.to_bytes(10, byteorder='big')
# nonce, proof = PoW(signed, tx_id_bytes)

# input_data = transaction_proto.InputData(
#     nonce = random.randint(1000, 5000),
#     block_height = block_height,
#     order_submission = order_data
# )
# serialised = input_data.SerializeToString()

# request = core_proto.SubmitTransactionRequest(
#     tx=transaction_proto.Transaction(
#         input_data=serialised,
#         signature=signature_proto.Signature(
#             value=signed,
#             algo="vega/ed25519",
#             version=1,
#         ),
#         pub_key=pub_key,
#         version=2,
#         pow=transaction_proto.ProofOfWork(
#             tid=str(tx_id_bytes),
#             nonce=int.from_bytes(nonce, byteorder='big'),
#             hash_function="sha3_24_rounds"
#         )
#     ),
#     type=core_proto.SubmitTransactionRequest.Type.TYPE_ASYNC
# )

# vega.core_client().SubmitTransaction(request)

177 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [62]:
vega.stop()

In [127]:
%%timeit
vega.core_client().SubmitTransaction(request)

360 µs ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
